<a href="https://colab.research.google.com/github/harrymkwn/InfluenceAnalysis/blob/master/AllEkphrasisvec3negpos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from google.colab import  drive
drive.mount('/content/drive')
import nltk
from nltk.corpus import stopwords
nltk.download('all')
device_name = tf.test.gpu_device_name()
print(device_name)
df_train = pd.read_csv('/content/drive/MyDrive/InfluenceAnalysis/Newdata/train_clean.csv',names=['Tweet','Emotion'])
df_test = pd.read_csv('/content/drive/MyDrive/InfluenceAnalysis/Newdata/test_clean.csv',names=['Tweet','Emotion'])
df_train = df_train.sample(frac=1)
df_test = df_train.sample(frac=1)

Mounted at /content/drive
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...

In [2]:
df_train.head(1000)

,Tweet,Emotion
10339,lisibo oohh sorri so sad never notic the dol...,2
7843,chocoluvalm I am tri to figur that out right now,0
9444,anberlin welcom to seattl i realli wish i coul...,1
3088,dr_dinasadik forens mein kya hai ba rat te jaa...,0
5214,realli hate delay train especi minut delay tra...,0
...,...,...
7417,moossacr lol funni thing is i do not even kn...,0
9734,mit bookstor ha best book select but it the on...,2
7516,dsml do not depress me i have the motherinlaw ...,2
4300,pistol is the bee to busi to talk to hi buddi ...,2


In [3]:
df_test.Emotion.unique()

array(['0', '2', '1', '3', 'emotion'], dtype=object)

In [4]:
df_train.nunique()

Tweet      12370
Emotion        5
dtype: int64

In [5]:
category_dict = {'0':[1,0,0,0],'1':[0,1,0,0],'2':[0,0,1,0],'3':[0,0,0,1],'emotion':[1,0,0,0]}
train_data_tweet = [x.lower().split() for x in df_train['Tweet']]
train_data_cat = np.array([category_dict[x] for x in df_train['Emotion']])
test_data_tweet = [x.lower().split() for x in df_test['Tweet']]
test_data_cat = np.array([category_dict[x] for x in df_test['Emotion']])

data_tweet = train_data_tweet + test_data_tweet

In [6]:
# Parameters
Min_count = 0
Embedding_size = 100
Window_size = 5
Negative_sampling = 00

In [7]:
w2v_sg = Word2Vec(min_count=Min_count,
                     window=Window_size,
                     size=Embedding_size,
                     negative=Negative_sampling,sg=1)
w2v_sg.build_vocab(data_tweet)
w2v_sg.train(data_tweet, total_examples=w2v_sg.corpus_count, epochs=5)
w2v_cbow = Word2Vec(min_count=Min_count,
                     window=Window_size,
                     size=Embedding_size,
                     negative=Negative_sampling,sg=0)
w2v_cbow.build_vocab(data_tweet)
w2v_cbow.train(data_tweet, total_examples=w2v_cbow.corpus_count, epochs=5)

(1303719, 1724630)

In [8]:
vocab_sg = w2v_sg.wv.vocab
vocab_sg = [x for x in vocab_sg]
vocab_cbow = w2v_cbow.wv.vocab
vocab_cbow = [x for x in vocab_cbow] 
vocab_glove = {}
with open("/content/drive/My Drive/InfluenceAnalysis/glove/glove.twitter.27B.100d.txt", 'r', encoding="utf-8") as f:
  for line in f:
      values = line.split()
      word = values[0]
      vector = np.asarray(values[1:], "float32")
      vocab_glove[word] = vector

In [9]:
batch_size = 500
Max_input_size = max([len(x) for x in data_tweet])
mul_factor=1.5

In [10]:
positive = pd.read_csv('/content/drive/My Drive/InfluenceAnalysis/CodeMix/positive-words.csv')
positive = list(positive['0'][1:])
negative = pd.read_csv('/content/drive/My Drive/InfluenceAnalysis/CodeMix/negative-words.csv')
negative = list(negative['0'][2:])
pn_dict = {}
for i in range(len(positive)):
  pn_dict[positive[i]] = 1

for i in range(len(negative)):
  pn_dict[negative[i]] = -1

In [11]:
X_train = train_data_tweet
X_test = test_data_tweet
y_train = train_data_cat
y_test = test_data_cat 
X_train = [nltk.pos_tag(x) for x in X_train]
X_test = [nltk.pos_tag(x) for x in X_test]
X_train = [[(i[0],i[1],pn_dict.get(i[0],1)) for i in x] for x in X_train]
X_test = [[(i[0],i[1],pn_dict.get(i[0],1)) for i in x] for x in X_test]
result_table = [0,0,0,0,0,0,0,0,0]

In [12]:
print(X_train[0][0])

('lisibo', 'NN', 1)


In [13]:
def vec_gen(w2v,vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size,mul_factor):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      res=[]
      for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]:
        tweet = [(w2v.wv.get_vector(i[0]),i[1],i[2]) for i in x if i[0] in vocab]
        l=len(tweet)
        for i in range(l):
          tweet[i]=((tweet[i][0]*tweet[i][2]),tweet[i][1],tweet[i][2])
        for i in range(l):
          if tweet[i][1][:2]=='JJ':
            for z in range(max(i-2,0),min(i+3,l)):
              if tweet[z][1][:2]=='NN' or tweet[z][1][:2]=='VB':
                tweet[z]=((tweet[z][0]*tweet[i][0])*mul_factor/np.sqrt(tweet[i][0].dot(tweet[i][0])),tweet[z][1])
          if tweet[i][1][:2]=='RB':
            for z in range(max(i-2,0),min(i+3,l)):
              if tweet[z][1][:2]=='VB':
                tweet[z]=((tweet[z][0]*tweet[i][0])*mul_factor/np.sqrt(tweet[i][0].dot(tweet[i][0])),tweet[z][1])
        res+=[np.array([x[0] for x in tweet])]
      temp = np.array([np.pad(z.flatten(),(0,Max_input_size*Embedding_size-len(z.flatten()))).reshape(Max_input_size,Embedding_size) for z in res])
      tempres = data_cat[k*(batch_size):(k+1)*(batch_size)]
      yield (temp,tempres)          
      
def glove_gen(vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size,mul_factor):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      res=[]
      for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]:
        tweet = [(vocab[i[0]],i[1],i[2]) for i in x if i[0] in vocab.keys()]
        l=len(tweet)
        for i in range(l):
          tweet[i]=((tweet[i][0]*tweet[i][2]),tweet[i][1],tweet[i][2])
        for i in range(l):
          if tweet[i][1][:2]=='JJ':
            for z in range(max(i-2,0),min(i+3,l)):
              if tweet[z][1][:2]=='NN' or tweet[z][1][:2]=='VB':
                tweet[z]=((tweet[z][0]*tweet[i][0])*mul_factor/np.sqrt(tweet[i][0].dot(tweet[i][0])),tweet[z][1])
          if tweet[i][1][:2]=='RB':
            for z in range(max(i-2,0),min(i+3,l)):
              if tweet[z][1][:2]=='VB':
                tweet[z]=((tweet[z][0]*tweet[i][0])*mul_factor/np.sqrt(tweet[i][0].dot(tweet[i][0])),tweet[z][1])
        res+=[np.array([x[0] for x in tweet])]
      temp = np.array([np.pad(z.flatten(),(0,Max_input_size*Embedding_size-len(z.flatten()))).reshape(Max_input_size,Embedding_size) for z in res])
      tempres = data_cat[k*(batch_size):(k+1)*(batch_size)]
      yield (temp,tempres)

In [14]:
epochs = 20

In [15]:
model_sg = tf.keras.Sequential()
model_sg.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg.add(tf.keras.layers.LSTM(50))
model_sg.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_sg.fit_generator(generator = vec_gen(w2v_sg,vocab_sg,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_sg.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[0]=history[1]
print("SG_LSTM")


Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/20
24/24 [==============================] - 5s 224ms/step - loss: 1.2700 - accuracy: 0.5726
Epoch 2/20
24/24 [==============================] - 5s 229ms/step - loss: 1.0381 - accuracy: 0.5869
Epoch 3/20
24/24 [==============================] - 5s 228ms/step - loss: 0.9865 - accuracy: 0.5869
Epoch 4/20
24/24 [==============================] - 6s 232ms/step - loss: 0.9848 - accuracy: 0.5869
Epoch 5/20
24/24 [==============================] - 5s 229ms/step - loss: 0.9848 - accuracy: 0.5869
Epoch 6/20
24/24 [==============================] - 5s 228ms/step - loss: 0.9848 - accuracy: 0.5869
Epoch 7/20
24/24 [==============================] - 5s 226ms/step - loss: 0.9848 - accuracy: 0.5869
Epoch 8/20
24/24 [==============================] - 5s 229ms/step - loss: 0.9847 - accuracy: 0.5869
Epoch 9/20
24/24 [==============================] - 6s 229ms/step - loss: 0.9847 - accuracy: 0.5869
Epoch 10/20
24/24 [=====

In [16]:
model_sg_bi = tf.keras.Sequential()
model_sg_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_sg_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_sg_bi.fit_generator(generator = vec_gen(w2v_sg,vocab_sg,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_sg_bi.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[1]=history[1]
print("SG_BI")

Epoch 1/20
24/24 [==============================] - 5s 226ms/step - loss: 1.2254 - accuracy: 0.5769
Epoch 2/20
24/24 [==============================] - 5s 228ms/step - loss: 0.9962 - accuracy: 0.5869
Epoch 3/20
24/24 [==============================] - 6s 234ms/step - loss: 0.9862 - accuracy: 0.5869
Epoch 4/20
24/24 [==============================] - 6s 231ms/step - loss: 0.9848 - accuracy: 0.5869
Epoch 5/20
24/24 [==============================] - 6s 231ms/step - loss: 0.9846 - accuracy: 0.5869
Epoch 6/20
24/24 [==============================] - 5s 228ms/step - loss: 0.9846 - accuracy: 0.5869
Epoch 7/20
24/24 [==============================] - 5s 226ms/step - loss: 0.9845 - accuracy: 0.5869
Epoch 8/20
24/24 [==============================] - 5s 227ms/step - loss: 0.9844 - accuracy: 0.5869
Epoch 9/20
24/24 [==============================] - 5s 225ms/step - loss: 0.9843 - accuracy: 0.5869
Epoch 10/20
24/24 [==============================] - 5s 225ms/step - loss: 0.9843 - accuracy: 0.5869

In [17]:
model_sg_rnn = tf.keras.Sequential()
model_sg_rnn.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg_rnn.add(tf.keras.layers.SimpleRNN(50))
model_sg_rnn.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_sg_rnn.fit_generator(generator = vec_gen(w2v_sg,vocab_sg,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_sg_rnn.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[2]=history[1]
print("SG_RNN")

Epoch 1/20
24/24 [==============================] - 6s 238ms/step - loss: 1.1003 - accuracy: 0.5621
Epoch 2/20
24/24 [==============================] - 6s 240ms/step - loss: 0.9872 - accuracy: 0.5869
Epoch 3/20
24/24 [==============================] - 6s 240ms/step - loss: 0.9854 - accuracy: 0.5869
Epoch 4/20
24/24 [==============================] - 6s 236ms/step - loss: 0.9852 - accuracy: 0.5869
Epoch 5/20
24/24 [==============================] - 6s 238ms/step - loss: 0.9852 - accuracy: 0.5869
Epoch 6/20
24/24 [==============================] - 6s 237ms/step - loss: 0.9852 - accuracy: 0.5869
Epoch 7/20
24/24 [==============================] - 6s 240ms/step - loss: 0.9852 - accuracy: 0.5869
Epoch 8/20
24/24 [==============================] - 6s 239ms/step - loss: 0.9852 - accuracy: 0.5869
Epoch 9/20
24/24 [==============================] - 6s 238ms/step - loss: 0.9852 - accuracy: 0.5869
Epoch 10/20
24/24 [==============================] - 6s 241ms/step - loss: 0.9852 - accuracy: 0.5869

In [ ]:
model_cbow = tf.keras.Sequential()
model_cbow.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow.add(tf.keras.layers.LSTM(50))
model_cbow.add(tf.keras.layers.Dense(4, activation='softmax'))
model_cbow.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_cbow.fit_generator(generator = vec_gen(w2v_cbow,vocab_cbow,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_cbow.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[3]=history[1]
print("CBOW_LSTM")

In [ ]:
model_cbow_bi = tf.keras.Sequential()
model_cbow_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_cbow_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_cbow_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_cbow_bi.fit_generator(generator = vec_gen(w2v_cbow,vocab_cbow,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_cbow_bi.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[4]=history[1]
print("CBOW_BI")

In [ ]:
model_cbow_rnn = tf.keras.Sequential()
model_cbow_rnn.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow_rnn.add(tf.keras.layers.SimpleRNN(50))
model_cbow_rnn.add(tf.keras.layers.Dense(4, activation='softmax'))
model_cbow_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_cbow_rnn.fit_generator(generator = vec_gen(w2v_cbow,vocab_cbow,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_cbow_rnn.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[5]=history[1]
print("CBOW_RNN")

In [ ]:
epochs=40
Embedding_size=100

In [ ]:
model_glove = tf.keras.Sequential()
model_glove.add(tf.keras.Input(shape=(Max_input_size,100)))
model_glove.add(tf.keras.layers.LSTM(50))
model_glove.add(tf.keras.layers.Dense(4, activation='softmax'))
model_glove.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_glove.fit_generator(generator = glove_gen(vocab_glove,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_glove.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[6]=history[1]
print("GLOVE_LSTM")


In [ ]:
model_glove_bi = tf.keras.Sequential()
model_glove_bi.add(tf.keras.Input(shape=(Max_input_size,100)))
model_glove_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_glove_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_glove_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_glove_bi.fit_generator(generator = glove_gen(vocab_glove,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_glove_bi.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[7]=history[1]
print("GLOVE_BI")

In [ ]:
model_glove_rnn = tf.keras.Sequential()
model_glove_rnn.add(tf.keras.Input(shape=(Max_input_size,100)))
model_glove_rnn.add(tf.keras.layers.SimpleRNN(50))
model_glove_rnn.add(tf.keras.layers.Dense(4, activation='softmax'))
model_glove_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_glove_rnn.fit_generator(generator = glove_gen(vocab_glove,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_glove_rnn.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[8]=history[1]
print("GLOVE_RNN")

In [ ]:
model_sg1 = tf.keras.Sequential()
model_sg1.add(tf.keras.Input(shape=(Max_input_size,100)))
model_sg1.add(tf.keras.layers.LSTM(64))
model_sg1.add(tf.keras.layers.Dropout(0.2))
model_sg1.add(tf.keras.layers.Dense(64,activation='relu'))
model_sg1.add(tf.keras.layers.Dropout(0.2))
model_sg1.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg1.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_sg1.fit_generator(generator = vec_gen(w2v_sg,vocab_sg,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_sg1.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table+=[history[1]]
print("SG_LSTM")

In [ ]:
print("\tLstm\t\t\tBiLstm\t\t\tRnn")
print("sg",result_table[:3])
print("cbow",result_table[3:6])
print("glove",result_table[6:9])

In [ ]:
print(result_table[9])